# Example Jupyter notebook to work with the data

# Read in and plot the Apollo 12 Grade A catalog

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

In [2]:
def spectrogram_plot(tr_times_filt, tr_data_filt, t, f, sxx, cm):
    fig = plt.figure(figsize=(10, 10))
    ax2 = plt.subplot(1, 1, 1)
    vals = ax2.pcolormesh(t, f, sxx, cmap=cm.jet, vmax=5e-17)
    # ax2.set_xlim([min(tr_times_filt),max(tr_times_filt)])
    ax2.set_xlabel(f'Time (Day Hour:Minute)', fontweight='bold')
    ax2.set_ylabel('Frequency (Hz)', fontweight='bold')
    cbar = plt.colorbar(vals, orientation='horizontal')
    cbar.set_label('Power ((m/s)^2/sqrt(Hz))', fontweight='bold')

In [3]:
def prepare_event_data_dict(list_of_events):
    list_of_event_ids = []
    event_data_dict = {}
    for event_idx in range(len(list_of_events)):
        event_data = list_of_events.iloc[event_idx]
        event_filename = event_data['filename']
        event_time_abs = event_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)']
        event_time_rel = event_data['time_rel(sec)']
        event_id = event_data['evid']
        event_type = event_data['mq_type']
        list_of_event_ids.append(event_id)
        event_data_dict[event_id] = {'filename': event_filename, 'time_abs': event_time_abs, 'time_rel': event_time_rel, 'type': event_type}
    return list_of_event_ids, event_data_dict

def average_in_second_axis(data, factor):
    if data.shape[1] % factor == 0:
        return np.mean(data.reshape(data.shape[0], -1, factor), axis=2)
    else:
        num_full_groups = data.shape[1] // factor
        full_groups = np.mean(data[:, :num_full_groups * factor].reshape(data.shape[0], -1, factor), axis=2)
        remainder_group = np.mean(data[:, num_full_groups * factor:], axis=1, keepdims=True)
        return np.concatenate((full_groups, remainder_group), axis=1)

def apply_filter(st, minfreq, maxfreq):
    st_filt = st.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_data_filt = tr_filt.data

    f, t, sxx = signal.spectrogram(tr_data_filt, tr_filt.stats.sampling_rate)
    return f, t, sxx

In [4]:
event_list_file = './data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
list_of_events = pd.read_csv(event_list_file)
print(list_of_events)

                                  filename time_abs(%Y-%m-%dT%H:%M:%S.%f)  \
0   xa.s12.00.mhz.1970-01-19HR00_evid00002     1970-01-19T20:25:00.000000   
1   xa.s12.00.mhz.1970-03-25HR00_evid00003     1970-03-25T03:32:00.000000   
2   xa.s12.00.mhz.1970-03-26HR00_evid00004     1970-03-26T20:17:00.000000   
3   xa.s12.00.mhz.1970-04-25HR00_evid00006     1970-04-25T01:14:00.000000   
4   xa.s12.00.mhz.1970-04-26HR00_evid00007     1970-04-26T14:29:00.000000   
..                                     ...                            ...   
71  xa.s12.00.mhz.1974-10-14HR00_evid00156     1974-10-14T17:43:00.000000   
72  xa.s12.00.mhz.1975-04-12HR00_evid00191     1975-04-12T18:15:00.000000   
73  xa.s12.00.mhz.1975-05-04HR00_evid00192     1975-05-04T10:05:00.000000   
74  xa.s12.00.mhz.1975-06-24HR00_evid00196     1975-06-24T16:03:00.000000   
75  xa.s12.00.mhz.1975-06-26HR00_evid00198     1975-06-26T03:24:00.000000   

    time_rel(sec)       evid    mq_type  
0         73500.0  evid00002  imp

In [5]:
list_of_event_ids, event_data_dict = prepare_event_data_dict(list_of_events)

In [6]:
from scipy import signal
from matplotlib import cm

In [37]:
overlap = 0.25 #hours
window_length = 1 #hours

decimation_factor = 3
all_spectrograms = []

training_data_dir = './data/lunar/training/data/S12_GradeA/'
list_of_files = os.listdir(training_data_dir)
list_of_files = [file for file in list_of_files if file.endswith('.mseed')]
tr_data = None
for file_idx in range(len(list_of_files)):
    # print(file)
    current_event_date = list_of_files[file_idx].split('.')[4][:10]
    current_event_date = datetime.strptime(current_event_date, "%Y-%m-%d")
    if file_idx == len(list_of_files) - 1:
        days_difference = -1
    else:
        next_event_date = list_of_files[file_idx+1].split('.')[4][:10]
        next_event_date = datetime.strptime(next_event_date, "%Y-%m-%d")
        print(current_event_date, next_event_date, end=' ')

        delta = next_event_date - current_event_date
        days_difference = delta.days
        print(days_difference)

    mseed_file_path = f'{training_data_dir}{list_of_files[file_idx]}'
    st = read(mseed_file_path)
    sampling_rate = st[0].stats.sampling_rate
    tr = st.traces[0].copy()
    tr_times = tr.times()
    if tr_data is None:
        tr_data = tr.data
    else:
        tr_data = np.concatenate((tr_data, tr.data))
    print(tr_data.shape)


    if days_difference == 1:
        continue
    else:
        iterator = 0
        list_of_spectrograms = []
        end_of_file = False
        while not end_of_file:
            tmp_data = tr_data[iterator:iterator+int(window_length*3600*sampling_rate)]
            tmp_times = tr_times[iterator:iterator+int(window_length*3600*sampling_rate)]
            if len(tmp_data) < int(window_length*3600*sampling_rate):
                tmp_data = tr_data[-int(window_length*3600*sampling_rate):]
                tmp_times = tr_times[-int(window_length*3600*sampling_rate):]
                end_of_file = True
            # print(len(tmp_data), len(tmp_times))
            iterator += int(overlap*3600*sampling_rate)
            tmp_data_undersample = signal.decimate(tmp_data, decimation_factor, axis=0, zero_phase=True)
            _, _, sxx = signal.spectrogram(tmp_data_undersample, sampling_rate/decimation_factor, nfft=128, nperseg=128)
            list_of_spectrograms.append(sxx)
            # print(sxx.shape)
            # f, t, sxx = signal.spectrogram(tmp_data, sampling_rate)
            # print(sxx.shape)
            # spectrogram_plot(tmp_times, tmp_data, t, f, sxx, cm)
            # break
        tr_data = None
        # print(len(list_of_spectrograms))
        np_arr = np.array(list_of_spectrograms)
        all_spectrograms.append(np_arr)
        print(np_arr.shape)
all_spectrograms = np.concatenate(all_spectrograms, axis=0)
    # print(sxx.shape)
    # spectrogram_plot(tr_times, tr_data, t, f, sxx, cm)

    # print(sxx.shape)
    # sxx_avg = average_undersampling(sxx, average_window_length)
    # print(sxx_avg.shape)

    
    # f,t,sxx = apply_filter(st, minfreq, maxfreq)

    # print(tr_times, tr_data.shape)

    # truncate_size = len(tr_data) - (len(tr_data) % average_window_length)
    # truncated_arr = tr_data[:truncate_size]

    # avg_data = tr_data.reshape(-1, average_window_length)
    # avg_data = avg_data.mean(axis=1)
    # print(avg_data.shape)
    # print(tr_data.shape)
    # tr_data_avg = average_undersampling(tr_data, average_window_length)
    # tr_data_avg = signal.resample(tr_data, len(tr_data)//average_window_length)
    # tr_data_avg = tr_data
    # print(tr_data_avg.shape)

1970-01-19 00:00:00 1970-03-25 00:00:00 65
(572415,)
(94, 65, 70)
1970-03-25 00:00:00 1970-03-26 00:00:00 1
(572411,)
1970-03-26 00:00:00 1970-04-25 00:00:00 30
(1144822,)
(190, 65, 70)
1970-04-25 00:00:00 1970-04-26 00:00:00 1
(572415,)
1970-04-26 00:00:00 1970-06-15 00:00:00 50
(1144826,)
(190, 65, 70)
1970-06-15 00:00:00 1970-06-26 00:00:00 11
(572418,)
(94, 65, 70)
1970-06-26 00:00:00 1970-07-20 00:00:00 24
(572423,)
(94, 65, 70)
1970-07-20 00:00:00 1970-07-20 00:00:00 0
(572411,)
(94, 65, 70)
1970-07-20 00:00:00 1970-09-26 00:00:00 68
(572411,)
(94, 65, 70)
1970-09-26 00:00:00 1970-10-24 00:00:00 28
(572423,)
(94, 65, 70)
1970-10-24 00:00:00 1970-11-12 00:00:00 19
(572420,)
(94, 65, 70)
1970-11-12 00:00:00 1970-12-11 00:00:00 29
(572411,)
(94, 65, 70)
1970-12-11 00:00:00 1970-12-27 00:00:00 16
(572415,)
(94, 65, 70)
1970-12-27 00:00:00 1970-12-31 00:00:00 4
(572423,)
(94, 65, 70)
1970-12-31 00:00:00 1971-01-15 00:00:00 15
(572411,)
(94, 65, 70)
1971-01-15 00:00:00 1971-01-28 00:00

In [38]:
print(all_spectrograms.shape)

(7103, 65, 70)
